In [18]:
#Revisión de que spacy este instalado
import spacy
print(spacy.__version__)


ModuleNotFoundError: No module named 'spacy'

In [15]:
#Llamamos a las librerias que vamos a necesitar
import spacy
import pathlib
import pandas as pd
from collections import Counter
from spacy import displacy
from spacy.matcher import Matcher
import altair as alt


ModuleNotFoundError: No module named 'spacy'

In [13]:
!python -m spacy download es_core_news_sm

c:\Users\focus\OneDrive\Escritorio\Maestría\Desarrollo de proyecto I\ProyectoFinal_MCD-DP1\.venv\Scripts\python.exe: No module named spacy


In [14]:
import es_core_news_sm
nlp = es_core_news_sm.load()

ModuleNotFoundError: No module named 'es_core_news_sm'

In [36]:
# cargar un modelo
nlp = spacy.load("es_core_news_sm")

In [37]:

datos = pd.read_csv('../results/all_amazon_reviews.csv')

In [38]:
print (datos)


           asin         star_rating  \
0    B0CJNQBKX9  5.0 de 5 estrellas   
1    B0CJNQBKX9  5.0 de 5 estrellas   
2    B0C47PZ6HR  4.0 de 5 estrellas   
3    B0C47PZ6HR  4.0 de 5 estrellas   
4    B0C47PZ6HR  5.0 de 5 estrellas   
..          ...                 ...   
375  B0BCMKMMK1           No rating   
376  B0BCMKMMK1           No rating   
377  B0BRBV8L6X  4.0 de 5 estrellas   
378  B0BRBV8L6X  5.0 de 5 estrellas   
379  B0BRBV8L6X  5.0 de 5 estrellas   

                                                 title  \
0                EXCELENTE NITIDEZ Y CALIDAD DE IMAGEN   
1                                   Excelente producto   
2                                       Llegó sin base   
3                                                BUENA   
4                                        Bien producto   
..                                                 ...   
375                                         Be careful   
376  Only network device in my home that won't conn...   
377      

In [39]:
datos['review_text'] = datos['review_text'].fillna('') 

In [40]:
datos['doc'] = datos['review_text'].apply(lambda x: nlp(x) if isinstance(x, str) else '')

In [41]:
for doc in datos['doc']:
    for token in doc:
        print(token.text, token.lemma_, token.pos_, token.dep_)

Me yo PRON obj
fascino fascinar VERB ROOT
por por ADP case
la el DET det
versatilidad versatilidad NOUN obl
de de ADP case
sus su DET det
contrastes contraste NOUN nmod
, , PUNCT punct
la el DET det
verdad verdad NOUN nsubj
no no ADV advmod
había haber AUX aux
trabajado trabajar VERB advcl
con con ADP case
la el DET det
marca marca NOUN obl
pero pero CCONJ cc
se él PRON expl:pass
me yo PRON obj
hace hacer VERB conj
muy mucho ADV advmod
funcional funcional ADJ iobj
, , PUNCT punct
de de ADP case
buena buen ADJ amod
calidad calidad NOUN obl
y y CCONJ cc
excelente excelente ADJ amod
relación relación NOUN conj
precio precio NOUN obj
- - PUNCT punct
beneficio beneficio NOUN obj
. . PUNCT punct
muy mucho ADV advmod
recomendable recomendable ADJ advmod
volvería volver VERB ROOT
a a ADP mark
comprar comprar VERB xcomp
sin sin ADP case
dudas duda NOUN obl
. . PUNCT punct
Excelente excelente ADJ amod
calidad calidad NOUN nsubj
de de ADP case
imagenMe imagenMe PROPN nmod
gustó gustar VERB ROOT
m

In [42]:
displacy.render(doc, style="dep", jupyter=True)


In [4]:
!pip install altair


     ---------------------------------------- 0.0/133.1 kB ? eta -:--:--
     ---------------------------------------- 0.0/133.1 kB ? eta -:--:--
     --- ------------------------------------ 10.2/133.1 kB ? eta -:--:--
     -------- ---------------------------- 30.7/133.1 kB 640.0 kB/s eta 0:00:01
     ----------- ------------------------- 41.0/133.1 kB 487.6 kB/s eta 0:00:01
     ----------- ------------------------- 41.0/133.1 kB 487.6 kB/s eta 0:00:01
     --------------------------------- -- 122.9/133.1 kB 599.1 kB/s eta 0:00:01
     ------------------------------------ 133.1/133.1 kB 522.8 kB/s eta 0:00:00
     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ------------------------- ------------ 41.0/61.2 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 61.2/61.2 kB 822.1 kB/s eta 0:00:00
     ---------------------------------------- 0.0/55.8 kB ? eta -:--:--
     ------- -------------------------------- 10.2/55.8 kB ? eta -:--:--
   

In [6]:
comentarios_por_asin = datos.groupby('asin')['review_text'].apply(lambda x: ' '.join(x)).reset_index()

# Contar el número de comentarios por 'asin'
conteo_por_asin = datos['asin'].value_counts().reset_index()
conteo_por_asin.columns = ['asin', 'conteo']

# Crear una gráfica de barras con Altair
grafica = alt.Chart(conteo_por_asin).mark_bar(
    color='steelblue',  # Color de las barras
    size=30  # Ancho de las barras
).encode(
    x='asin:N',
    y='conteo:Q',
    tooltip=['asin:N', 'conteo:Q']
).properties(
    title='Número de comentarios por ASIN'
)

# Mostrar la gráfica
grafica

NameError: name 'datos' is not defined